Problem 1

In [3]:
import numpy as np

In [4]:
def generate_normal_sequence(n, means):
    """
    Generate a sequence of n independent normal random variables.
    Each random variable has variance 1 and mean depending on the current state.
    :param n: Number of random variables
    :param means: A list of possible means [a0, a1, a2, a3] corresponding to states {0, 1, 2, 3}
    :return: A list of generated random variables and their states
    """
    states = [0, 1, 2, 3]  # Possible states
    state_probs = [(k + 1) / 10 for k in states]  # Transition probabilities
    
    sequence = []
    current_state = np.random.choice(states, p=state_probs)  # Start in a random state
    for _ in range(n):
        mean = means[current_state]
        value = np.random.normal(loc=mean, scale=1)  # Generate normal random variable
        sequence.append((value, current_state))
        
        # Transition to next state
        current_state = np.random.choice(states, p=state_probs)
    
    return sequence

In [5]:
n = 100  # Number of variables
means = [0, 1, 2, 3]  # Means corresponding to states {0, 1, 2, 3}
generated_sequence = generate_normal_sequence(n, means)
print("Generated Sequence:")
for value, state in generated_sequence[:10]:  # Print first 10 for brevity
    print(f"Value: {value:.2f}, State: {state}")

Generated Sequence:
Value: 1.97, State: 3
Value: 0.91, State: 0
Value: 1.55, State: 0
Value: 1.77, State: 1
Value: -0.32, State: 2
Value: 0.69, State: 0
Value: 3.08, State: 3
Value: 1.53, State: 1
Value: -0.57, State: 0
Value: 2.38, State: 3


Problem 2

In [6]:
def self_consistency_algorithm(sequence, tolerance=0.001, max_iter=1000):
    """
    Estimate the probabilities p_k(k) and parameters a_k using a self-consistency algorithm.
    :param sequence: Generated sequence of random variables and their states
    :param tolerance: Convergence threshold
    :param max_iter: Maximum number of iterations
    :return: Estimated probabilities and means
    """
    states = [0, 1, 2, 3]
    n = len(sequence)
    values, observed_states = zip(*sequence)
    values = np.array(values)
    
    # Initialize parameters
    p_k = np.ones(len(states)) / len(states)  # Equal probabilities
    a_k = np.random.uniform(low=0, high=3, size=len(states))  # Random initialization of means
    
    for iteration in range(max_iter):
        prev_a_k = a_k.copy()
        prev_p_k = p_k.copy()
        
        # E-step: Compute responsibilities (posterior probabilities)
        responsibilities = np.zeros((n, len(states)))
        for i, state in enumerate(states):
            responsibilities[:, i] = (
                p_k[state] * np.exp(-(values - a_k[state])**2 / 2)
            )
        responsibilities /= responsibilities.sum(axis=1, keepdims=True)
        
        # M-step: Update p_k and a_k
        for state in states:
            weight = responsibilities[:, state].sum()
            a_k[state] = (responsibilities[:, state] @ values) / weight
            p_k[state] = weight / n
        
        # Check for convergence
        if np.max(np.abs(a_k - prev_a_k)) < tolerance and np.max(np.abs(p_k - prev_p_k)) < tolerance:
            print(f"Converged in {iteration} iterations.")
            break
    
    return p_k, a_k

In [7]:
estimated_p_k, estimated_a_k = self_consistency_algorithm(generated_sequence)
print("\nEstimated Probabilities p_k(k):", estimated_p_k)
print("Estimated Means a_k:", estimated_a_k)

Converged in 166 iterations.

Estimated Probabilities p_k(k): [0.01413448 0.30820252 0.49466253 0.18300047]
Estimated Means a_k: [-1.85138747  1.11260641  3.01263097  1.45973952]


Problem 3

In [8]:
def analyze_algorithm(n_list, initial_p_k, initial_a_k):
    """
    Analyze the behavior of the algorithm for different n and initial estimates.
    :param n_list: List of sequence lengths to analyze
    :param initial_p_k: Initial estimates of probabilities
    :param initial_a_k: Initial estimates of means
    """
    results = []
    for n in n_list:
        sequence = generate_normal_sequence(n, initial_a_k)
        p_k, a_k = self_consistency_algorithm(sequence)
        results.append((n, p_k, a_k))
    
    # Print results
    for n, p_k, a_k in results:
        print(f"\nFor n = {n}:")
        print("Estimated Probabilities p_k(k):", p_k)
        print("Estimated Means a_k:", a_k)

In [9]:
n_list = [50, 100, 200]
initial_p_k = [0.25, 0.25, 0.25, 0.25]  # Equal probabilities
initial_a_k = [0, 1, 2, 3]  # True means
analyze_algorithm(n_list, initial_p_k, initial_a_k)

Converged in 71 iterations.
Converged in 59 iterations.
Converged in 58 iterations.

For n = 50:
Estimated Probabilities p_k(k): [0.44962906 0.16422291 0.18935442 0.19679361]
Estimated Means a_k: [3.09854634 2.68998974 0.91013484 0.91013484]

For n = 100:
Estimated Probabilities p_k(k): [0.18666231 0.31926144 0.30678492 0.18729133]
Estimated Means a_k: [1.74533275 0.40930104 2.83546128 1.77696971]

For n = 200:
Estimated Probabilities p_k(k): [0.21618018 0.2877233  0.28121221 0.21488431]
Estimated Means a_k: [1.79977677 1.1552972  3.65663312 1.80823543]
